### Getting Started with MAF - Creating and Chatting with our Agent (At Microsoft Foundry Level)

### Installing Required Libraries

In [ ]:
%pip install agent-framework-azure-ai==1.0.0b251120 python-dotenv azure-ai-projects==2.0.0b1

### Setting Up the Environment

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")

### Creating the Foundry Client and Agent

In [ ]:
from azure.ai.projects.aio import AIProjectClient
from azure.identity import DefaultAzureCredential
from agent_framework_azure_ai import AzureAIClient

# creating an AzureAIProjectClient
project_client = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=foundry_project_endpoint
)

# Creating an AzureAIAgentClient
agent_client = AzureAIClient(
    project_client=project_client,
    model_deployment_name=model_deployment_name
)

agent = agent_client.create_agent(
    name = "BatmanAgent",
    instructions = "You are Batman, the dark knight of Gotham City."
)

In [ ]:
query = "Who is the Joker?"
response = await agent.run(query)
print("Agent Response:", response)

### Chatting with the Agent - Non Streaming

In [ ]:
query = "Who is the Joker?"
response = await agent.run(query)
print("Agent Response:", response)

### Chatting with the Agent - Streaming

In [ ]:
# Running the agent with streaming responses
async def streaming_chat():
    async for update in agent.run_stream("Tell me something interesting about Gotham City in 10 points"):
        if update.text:
            print(update.text, end="", flush=True)
    print()  # New line after streaming is complete

await streaming_chat()

### Giving our Agent an Image

![joker_interrogation](./Assets/joker_interrogation_scene.png)

In [ ]:
from agent_framework import ChatMessage, TextContent, DataContent, Role

# Load image from local file
with open("./Assets/joker_interrogation_scene.png", "rb") as image_file:
    image_bytes = image_file.read()

# Create a message with text and image data
message = ChatMessage(
    role = Role.USER,
    contents = [
        TextContent(text = "Tell me about the incident in this image."),
        DataContent(
            data = image_bytes,
            media_type = "image/png"
        )
    ]
)

# Creating an async function for the entire image analysis run
async def analyze_image():
    response = await agent.run(message)
    print(f"Agent: {response.text}")

# invoke the async function
await analyze_image()